##### Copyright 2018 The TensorFlow Authors.

In [0]:
#@title Licensed under the Apache License, Version 2.0 (the "License"); { display-mode: "form" }
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Конкретные функции

<table class="tfo-notebook-buttons" align="left">
  <td><a target="_blank" href="https://www.tensorflow.org/guide/concrete_function"><img src="https://www.tensorflow.org/images/tf_logo_32px.png"> Посмотреть на TensorFlow.org</a></td>
  <td><a target="_blank" href="https://colab.research.google.com/github/tensorflow/docs/blob/master/site/en/guide/concrete_function.ipynb"><img src="https://www.tensorflow.org/images/colab_logo_32px.png"> Запустить в Google Colab</a></td>
  <td><a target="_blank" href="https://github.com/tensorflow/docs/blob/master/site/en/guide/concrete_function.ipynb"><img src="https://www.tensorflow.org/images/GitHub-Mark-32px.png"> Посмотреть источник на GitHub</a></td>
  <td><a href="https://storage.googleapis.com/tensorflow_docs/docs/site/en/guide/concrete_function.ipynb"><img src="https://www.tensorflow.org/images/download_logo_32px.png"> Скачать блокнот</a></td>
</table>


В руководстве по [AutoGraph и `tf.functions`](function.ipynb) вы видели, как использовать `tf.function` . Это руководство углубляется в детали:

- `tf.function` Tracing
- `tf.function` подписи
- Конкретные функции, генерируемые при трассировке:
    - Как получить к ним доступ
    - Как их использовать

Эти детали только становятся важными:

- Если у вас возникают проблемы с производительностью из-за нежелательной трассировки `tf.funcion` .
- When you need precise control over the TensorFlow Graphs generated by `tf.function`. For example for exporting the model to [TensorFlow Lite](https://tensorflow.org/lite/) using `tf.lite.Converter.from_concrete_functions`.


## Задний план

В TensorFlow 2 активное выполнение включено по умолчанию. Стремительное выполнение TensorFlow является императивной средой программирования, которая оценивает операции немедленно, без построения графиков. Операции возвращают значения вместо построения вычислительного графа для запуска позже. Вот [подробное руководство по исполнению](eager.ipynb) .

Обязательный запуск делает разработку и отладку более интерактивными, но не позволяет легко экспортировать.

API `tf.function` позволяет сохранять модели в виде графиков.

## терминология

В этом документе используется следующая терминология:

- **Подпись** - описание входов и выходов для набора операций.
- **Полиморфная функция** - вызываемый Python, который инкапсулирует несколько графиков конкретных функций за одним API.
- **Конкретная функция** - граф с единой сигнатурой.


## Настроить

In [0]:
import traceback
import textwrap

!pip install tf_nightly


In [0]:
import tensorflow as tf

## Создать функцию `tf.function`

Annotating a function with `tf.function` generates a *polymorphic function* containing those operations. All operations that are not annotated with `tf.function` will be evaluated with eager execution. The examples below show a quick example of `tf.function` usage.

In [0]:
@tf.function
def square(x):
  return x*x

In [0]:
square(2).numpy()

Помните, что синтаксис декоратора Python просто вызывает декоратор с декорированным объектом на входе:

In [0]:
def pow(x,y):
  return x ** y

pow = tf.function(pow)

In [0]:
pow(3,4).numpy()

### Присоедините метод `tf.function` к `tf.Module`

Функция `tf.function` может быть дополнительно сохранена как часть объекта `tf.Module` . Класс `tf.Module` предоставляет функции для отслеживания переменных и сохранения [контрольных точек](checkpoints.ipynb) и [моделей](saved_model.ipynb) .

Такие классы, как `keras.layers.Layer` и `keras.Model` являются подклассами модуля.

In [0]:
class Pow(tf.Module):
  def __init__(self, exponent):
    self.exponent = tf.Variable(exponent, dtype = tf.float32, name='Pow/exponent')

  @tf.function
  def __call__(self, x):
    return x ** self.exponent

In [0]:
pow = Pow(3)

In [0]:
pow.variables

In [0]:
pow(tf.constant(2.0)).numpy()

In [0]:
pow.exponent.assign(4)
pow(tf.constant(2.0)).numpy()

In [0]:
tf.saved_model.save(pow, 'pow')

In [0]:
reloaded_pow = tf.saved_model.load('pow')

In [0]:
reloaded_pow(tf.constant(3.0)).numpy()

### Назначьте функцию `tf.function` как атрибут

Если вы назначите `tf.Module` или `tf.function` в качестве атрибута модуля, он также будет сериализован:

In [0]:
mod = tf.Module()
mod.increment_by = tf.Variable(2.0)

@tf.function
def increment(x):
  return x+mod.increment_by

mod.inc = increment
mod.inc(tf.constant(1.0)).numpy()

In [0]:
mod.cube = Pow(3)
mod.cube(tf.constant(2.0)).numpy()

In [0]:
mod.variables

In [0]:
tf.saved_model.save(mod, 'mod')
reloaded_mod = tf.saved_model.load('mod')

In [0]:
reloaded_mod.inc(4.0).numpy()

In [0]:
reloaded_mod.cube(4.0).numpy()

### Совместимость с `tf.keras`

Keras classes like `keras.Model` and `keras.layers.Layer` are fully compatible with `tf.function` and `tf.Module`.

Например, построить простую модель:

In [0]:
linear = tf.keras.Sequential([tf.keras.layers.Dense(units=1, input_shape=[1])])
linear.compile(optimizer='adam', loss='mean_squared_error')
linear.fit(x=[-1, 0, 1, 2, 3, 4], y=[-3, -1, 1, 3, 5, 7], epochs=50, verbose=0)

In [0]:
linear(tf.constant([[1],[2]]))

Проверьте это переменные

In [0]:
linear.variables

Теперь прикрепите его к `tf.Module` :

In [0]:
module = tf.Module()
module.linear = linear

Модуль `tf.Module` также отслеживает `tf.Variable` :

In [0]:
module.variables

Модуль `tf.Module` также экспортирует содержимое модели `keras.Model` :

In [0]:
tf.saved_model.save(module,'module')

In [0]:
reloaded = tf.saved_model.load('module')

In [0]:
reloaded.linear([[1.0]])

## трассировка

Объекты, возвращаемые из функции `tf.function` являются полиморфными функциями. Они будут принимать объекты Python или `tf.Tensors` с любой формой или `tf.dtype` качестве входных данных.

In the background TensorFlow builds `tf.Graph`s representing the calculation. This graph is wrapped in a python callable: a concrete function. Each concrete function can only handle a single input signature.

`tf.function` отслеживает функцию python каждый раз, когда необходимо создать конкретную функцию. Самый простой способ увидеть, когда функция отслеживается, это добавить вызов print:


In [0]:
@tf.function
def mul(a, b):
  print('Tracing:\n    {a}\n    {b}\n'.format(a=a, b=b))
  return a*b

### Dtypes и формы

Если вы вызываете полиморфную функцию с двумя различными типами ввода, она будет прослеживаться один раз для каждого:

In [0]:
# Trace with ints
mul(tf.constant(2), tf.constant(3)).numpy()

In [0]:
# Trace with floats
mul(tf.constant(2.0), tf.constant(3.0)).numpy()

Когда вы вызываете его снова с теми же типами ввода, он отправляет существующей функции вместо трассировки:

In [0]:
# Call with ints again => no trace
mul(tf.constant(10), tf.constant(10))

Изменение размеров входов также запускает трассировку (установка `tf.function(experimental_relax_shapes=True)` может уменьшить это): 

In [0]:
# Trace with vectors
mul(tf.constant([1.0,3.0]), tf.constant(3.0)).numpy()

In [0]:
# Trace with different-sized vectors
mul(tf.constant([1.0,2.0,3.0, 4.0]), tf.constant(3.0))

### Неизменяемые объекты Python

Если вы передаете неизменный объект python, например, `int` , `str` или `tuple` в функцию `tf.function` , он выполняет трассировку для каждого *значения* этих объектов python.

This is useful to control what gets included in the `tf.Graph` (See: [The Autograph Guide](function.ipynb) for more details).


In [0]:
@tf.function
def mul(a, b):
  print('Tracing:\n    {a}\n    {b}\n'.format(a=a, b=b))
  return a*b

In [0]:
# Trace for a=3.0
mul(3.0, tf.constant(3.0)).numpy()

In [0]:
# Don't trace for a=3.0 the second time:
mul(3.0, tf.constant(3.0)).numpy()

Внимание: легко вызвать много следов, передав уникальные значения Python. Это может быть серьезной проблемой производительности. Часто передача значения `tf.Tensor` является решением.

Этот цикл отслеживает функцию для каждого уникального целого:

In [0]:
@tf.function
def power(a,b):
  print('Tracing "power": a={}'.format(a))
  return a**b

In [0]:
p = tf.constant(2)
for n in range(12):
  power(n,p)

На втором запуске отслеживалось каждое int, поэтому нет никакой трассировки:

In [0]:
p = tf.constant(2)
for n in range(12):
  power(n,p)

Чтобы избежать избыточного восстановления, обязательно передайте `tf.Tensor` вместо чисел или строк python:

In [0]:
p = tf.constant(2)
for n in tf.range(12):
  power(n,p)

Чтобы вообще отключить трассировку, передайте подпись декоратору `tf.function` :

In [0]:
@tf.function(input_signature=(
    tf.TensorSpec(shape=[], dtype=tf.float32),
    tf.TensorSpec(shape=[], dtype=tf.float32),)
)
def power_with_sig(a,b):
  print('Tracing "power_with_sig"')
  return a**b

In [0]:
power_with_sig(3.0, 3.0).numpy()

In [0]:
try:
  power_with_sig(tf.constant([1.0,2.0,3.0]),tf.constant(3.0))
  assert False
except ValueError:
  traceback.print_exc(limit=1)

### Пример: выпадение

Возврат к определенным значениям дает вам контроль над тем, какой код генерируется `tf.function` .


In [0]:
class Dropout(tf.Module):
  def __init__(self, rate, name=None):
    super(Dropout, self).__init__(name)
    self.rate = tf.Variable(rate, dtype = tf.float32, trainable=False)

  @tf.function
  def __call__(self, x, training=True):
    print(textwrap.dedent("""
                          Tracing "Dropout":
                              training = {}
                              x = {}
                              name = {:s}
                          """.format(training, x, self.name)))
    if training:
      print('    - Train branch\n')
      mask = tf.random.uniform(x.shape) > self.rate
      return x * tf.cast(mask, tf.float32)/self.rate
    else:
      print('    - Test branch\n')
      return x

Создайте экземпляр этого простого слоя `Dropout` :

In [0]:
dropout = Dropout(0.5)

Когда вы в первый раз вызываете его с помощью Python `training=True` качестве входных данных, он отслеживает ветвь `training` :

In [0]:
dropout(tf.range(10, dtype=tf.float32), training=True).numpy()

Во второй раз не нужно пересматривать ветку:

In [0]:
dropout(tf.range(10, dtype=tf.float32), training=True).numpy()

Передача `training=False` запускает трассировку при первом запуске, поскольку это другое значение Python:

In [0]:
dropout(tf.range(10, dtype=tf.float32), training=False).numpy()

In [0]:
dropout(tf.range(10, dtype=tf.float32), training=False).numpy()

If you pass a `bool` tensor, it uses TensorFlow autograph rewrite the `if` to a `tf.cond`m and traces both branches:

In [0]:
dropout(tf.range(10, dtype=tf.float32), training=tf.constant(False)).numpy()

Это захватывает поток управления в одной конкретной функции.

In [0]:
 dropout(tf.range(10, dtype=tf.float32), training=tf.constant(True)).numpy()

In [0]:
dropout(tf.range(10, dtype=tf.float32), training=tf.constant(False)).numpy()

### Другие объекты Python

Поскольку сгенерированные `tf.Graphs` не могут содержать сложные объекты Python, они включаются путем трассировки и захвата переменных.

Функция `tf.function` запускает отдельную трассировку для каждого **экземпляра** . Таким образом, каждая трасса включает свои собственные переменные и может устанавливать свое поведение в зависимости от экземпляра.

Чаще всего используются методы Module, Layer или Module:

In [0]:
dropout_a = Dropout(0.5, name='dropout_a')

In [0]:
print(dropout_a(tf.range(10, dtype=tf.float32), True).numpy())
print(dropout_a(tf.range(10, dtype=tf.float32), True).numpy())

In [0]:
dropout_b = Dropout(0.5, name='dropout_b')

In [0]:
print(dropout_b(tf.range(10, dtype=tf.float32), True).numpy())
print(dropout_b(tf.range(10, dtype=tf.float32), True).numpy())

Но поведение на автономной функции `tf.function` .

In [0]:
@tf.function
def run(callable, x):
  print('Tracing "run":\n    callable = {}\n    x = {}\n'.format(callable, x))
  return callable(x)

In [0]:
def plus_1(x):
  return x+1

print(run(plus_1, tf.constant(2.0)).numpy())
print(run(plus_1, tf.constant(5.0)).numpy())

The tracing one `tf.function` can trigger tracing in another:

In [0]:
print(run(dropout, tf.range(10.0)).numpy())
print(run(dropout, tf.range(10.0)).numpy())

### Слабые ссылки

Осторожно: каждая трасса содержит только [слабую ссылку](https://docs.python.org/3/library/weakref.html) на любой `tf.Variable` . Если переменная не поддерживается другой ссылкой, трассировка может стать непригодной для использования.

Например, вот `tf.function` которая ссылается на `var` из прилагаемой области видимости:

In [0]:
@tf.function
def plus_var(x):
  print('Tracing "plus_var":\n    x = {}\n    var = {}\n\n'.format(x, var.name))
  return x + var

Проследить функцию с одной переменной:

In [0]:
var = tf.Variable(1, name="IntVar")
plus_var(tf.constant([1,2])).numpy()

И с другой переменной:

In [0]:
var = tf.Variable(2.0, name="FloatVar")
plus_var(tf.constant([2.0, 10.0])).numpy()

Это сработало, но из-за того, что у вас больше нет ссылки на `"IntVar"` , эта первая трассировка не работает:

In [0]:
try:
  plus_var(tf.constant([1,2])).numpy()
  assert False
except tf.errors.FailedPreconditionError:
  traceback.print_exc(limit=1)

## Доступ к конкретной функции

В предыдущем разделе вы видели условия для запуска нового следа полиморфной функции `tf.function` . Каждый след генерирует новую конкретную функцию.

Когда вы сохраняете `tf.Module` как `tf.saved_model` именно эти конкретные функции определяют `tf.Graph` . Вы не сохраняете функцию `tf.function` вы сохраняете конкретные функции, созданные путем трассировки.

Чтобы получить конкретную функцию из полиморфной функции `tf.function` вам нужно определить сигнатуру. Либо:

- Передайте `input_signature` в `tf.function` и вызовите метод `get_concrete_function()` .
- Передайте список `tf.TensorSpec` s `get_concrete_function` : `tf.TensorSpec(shape=[1], dtype=tf.float32)` .
- Передайте примерный тензор правильной формы и типа в `get_concrete_function` : `tf.constant(1., shape=[1])` .

В следующем примере показано , как определить `input_signature` параметр для `tf.function` .

#### Использование `input_signature`

Укажите входные тензоры в вызове функции `tf.function` как показано ниже. Эта `tf.function` может выполняться только для тензоров, которые соответствуют указанному подписи.

`None` в `shape` действует подстановочный знак. Так что эти `tf.TensroSpec` говорят "вектор float32 любой длины".

Этот шаблон может быть очень важен, если ожидается, что ваша `tf.function` будет обрабатывать последовательности разной длины или изображения разных размеров для каждого пакета (см., Например, инструкции [Transformer](../tutorials/text/transformer.ipynb) и [Deep Dream](../tutorials/generative/deepdream.ipynb) ).

In [0]:
@tf.function(input_signature=(
    tf.TensorSpec(shape=[None], dtype=tf.float32),
    tf.TensorSpec(shape=[None], dtype=tf.float32),)
)
def power_with_sig(a,b):
  print('Tracing "power_with_sig"\n')
  return a**b

Вызов `get_concrete_function` выполнит трассировку (при необходимости) и вернет конкретную функцию.

In [0]:
p = power_with_sig.get_concrete_function()
type(p)

In [0]:
p(tf.constant([2.0,3.0,4.0]), tf.constant([5.0,4.0,3.0])).numpy()

### Использование `get_concrete_function`

In [0]:
@tf.function
def power(a,b):
  print('Tracing "power"\n')
  return a**b

In [0]:
float_power = power.get_concrete_function(
  a = tf.TensorSpec(shape=[], dtype=tf.float32),
  b = tf.TensorSpec(shape=[], dtype=tf.float32))

In [0]:
float_power(tf.constant(3.0),tf.constant(3.0))

Помните, что вы также можете передавать тензоры в `get_concrete_function` , в этом случае она возвращает конкретную функцию, которая будет работать для этих входных данных:

In [0]:
row = tf.range(10)
col = tf.constant([[1],[2],[3]])

concrete_power = power.get_concrete_function(a = row, b = col)
concrete_power(row, col).numpy()

## Использование конкретной функции

Конкретная функция принимает только тензоры в качестве входных данных:

In [0]:
float_power(tf.constant(2.0), tf.constant(3.0)).numpy()

In [0]:
try:
  float_power(2.0,3.0)
  assert False
except (ValueError, TypeError):
  traceback.print_exc(limit=1)

Он также принимает только входы правильного dtype:

In [0]:
try:
  float_power(tf.constant(1),tf.constant(3))
  assert False
except tf.errors.InvalidArgumentError:
  traceback.print_exc(limit=1)

Но он попытается выполнить, даже если входные тензоры не соответствуют ожидаемой форме:

In [0]:
float_power(tf.constant([1.,2.,3.,4.,5.]),tf.constant(3.)).numpy()

In [0]:
try:
  float_power(tf.constant([1.,2.,3.]),tf.constant([4., 5.])).numpy()
  assert False
except tf.errors.InvalidArgumentError:  
  traceback.print_exc(limit=1)

Изучив конкретную функцию, вы можете увидеть ее входы и выходы:

In [0]:
print(float_power.structured_input_signature)
print(float_power.structured_outputs)

## Объекты Python в подписи

Как вы видели при трассировке, каждый объект python генерирует новую трассировку. Конкретные функции представляют собой единый `tf.Graph` , они не выполняют никакого восстановления. Когда вы вызываете `get_concrete_function` с объектом python в качестве одного из аргументов, объект **привязывается** к функции.

In [0]:
cube = power.get_concrete_function(
    a = tf.TensorSpec([], dtype=tf.float32),
    b = 3.0)

Эта функция `cube` больше не имеет аргумента `b` :

In [0]:
print(cube.structured_input_signature)

In [0]:
cube(tf.constant(10.0)).numpy()

Это очень похоже на способ, которым стандартные классы Python связывают методы, и в равной степени применяется при запуске `get_concrete_function` из метода:

In [0]:
class Greeter(object):
  def __init__(self, greeting):
    self.greeting = greeting

  def greet(self, who):
    return " ".join([self.greeting, who])

p = Greeter("Hello")
m = p.greet
print(m)

In [0]:
print(m("TensorFlow!"))

Когда у вас есть `tf.function` украшающая метод, применяются аналогичные правила:

In [0]:
class MyModel(tf.Module):
  def __init__(self, ins, outs):
    initializer = tf.initializers.GlorotNormal()
    self.W = tf.Variable(initializer([ins, outs]))
    self.B = tf.Variable(tf.zeros([outs], dtype = tf.float32))

  @tf.function
  def run(self, x):
    print('Tracing "MyModule":\n    x={}\n'.format(x))
    return tf.matmul(x, self.W)+self.B

In [0]:
mod = MyModel(ins=5, outs=3)

In [0]:
mod.run([[1.0,1.0,1.0, 1.0, 1.0]]).numpy()

Если вы вызываете метод `.get_concrete_function` , `self` автоматически связывается в качестве первого аргумента:

In [0]:
concrete_run = mod.run.get_concrete_function(x = tf.TensorSpec([None, None]))

In [0]:
concrete_run(tf.constant([[1.0,1.0,1.0, 1.0, 1.0],
                          [2.0,2.0,2.0, 2.0, 2.0]])).numpy()

Посмотрите, как `self` больше не является частью входной подписи:

In [0]:
print(concrete_run.structured_input_signature)
print(concrete_run.structured_outputs)

## Изменения для TensorFlow 2.3

Следующие изменения в настоящее время доступны в TensorFlow по ночам и будут доступны в TensorFlow 2.3.

### Проверка конкретных сигнатур функций

Печать `ConcreteFunction` отображает сводку его входных аргументов (с типами) и его выходного типа:

In [0]:
print(float_power)

Для полиморфных функций метод `pretty_printed_concrete_signatures()` может использоваться для отображения списка всех подписей, которые были отслежены до сих пор:

In [0]:
print(power.pretty_printed_concrete_signatures())

### Связанные аргументы

Как обсуждалось выше, когда создается конкретная функция, любые аргументы, для которых заданы значения, отличные от Tensor, *привязываются* к этим значениям. До TensorFlow 2.3 эти связанные аргументы были просто удалены из сигнатуры конкретной функции. Начиная с TensorFlow 2.3, они остаются в сигнатуре, но для их значения по умолчанию установлено связанное значение. Например, аргумент `b` имеет значение по умолчанию, равное `2` когда мы создаем следующую конкретную функцию:

In [0]:
square = power.get_concrete_function(a=tf.TensorSpec(None, tf.float32), b=2)
print(square)

При вызове `square` вы можете либо опустить `b` , либо явно установить для него значение, которое использовалось во время трассировки:

In [0]:
print(square(tf.constant(5.0)))  # ok: use default value
print(square(tf.constant(5.0), b=2))  # ok: explicit value == bound value

Установка привязанного аргумента в любое значение, отличное от его значения по умолчанию, является ошибкой:

In [0]:
try:
  print(square(tf.constant(10.0), b=4.0).numpy())  # error: explicit value != bound value
  assert False
except TypeError:
  traceback.print_exc(limit=1)

### Аргументы CompositeTensor

Начиная с TensorFlow 2.3, вы можете определить конкретные функции , которые принимают `CompositeTensor` s (например, `tf.RaggedTensor` , `tf.Dataset` итераторы и `tf.SparseTensor` ). Например:

In [0]:
ragged_power = power.get_concrete_function(
    a = tf.RaggedTensorSpec([None, None], dtype=tf.float32),
    b = tf.TensorSpec([], dtype=tf.float32)
)
print(ragged_power(tf.ragged.constant([[1.0, 2.0], [3.0]]),
                   tf.constant(2.0)))

### Вложенные аргументы

Начиная с TensorFlow 2.3, вы можете определить конкретные функции , которые принимают вложенную `dict` s, `list` s, `tuple` s, `namedtuple` сек и [`attr`](https://www.attrs.org/en/stable/) сек тензоров (или композитных тензоров). Например:

In [0]:
@tf.function
def sum_features(features):
  assert isinstance(features, dict)
  return sum(t for t in features.values())

features = {
    'a': tf.constant([[1.0], [2.0], [8.0]]),
    'b': tf.constant([[5.0], [6.0], [0.0]]),
    'c': tf.ragged.constant([[10.0], [20.0, 30.0, 40.0], [50.0]])
}
print(sum_features(features))

concrete_sum_features = sum_features.get_concrete_function(features)
print(concrete_sum_features(features))

Значение, передаваемое конкретной функции, должно иметь ту же структуру вложенности, которая использовалась при трассировке конкретной функции. Если вы передаете другую структуру, то возникает `TypeError` с информацией о принятой структуре и использованной структуре:

In [0]:
try:
  concrete_sum_features({'a': tf.constant([[1.0]])})
  assert False
except TypeError:
  traceback.print_exc(limit=1)